Based on 4chan NovelAILeaks (naifu)[src](https://boards.4channel.org/g/thread/89095460#p89097704)
maintained by : x1101 [src](https://github.com/DEX-1101)

In [ ]:
#@title ### 0. Check GPU working status

!nvidia-smi

In [2]:
#@title ### 1. Download Novel AI API backend, model
#@markdown If the download speed is too slow try restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
# config import (fix cant import 'rank_zero_only')
!gdown 1tINcyqr2o6wken6DhjcvAI6_NBKxfkQP -O /content/naifu/requirements.txt
!echo "Done."

/content
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libc-ares2
The following NEW packages will be installed:
  aria2 libc-ares2
0 upgraded, 2 newly installed, 0 to remove and 4 not upgraded.
Need to get 1,274 kB of archives.
After this operation, 4,912 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.14.0-1ubuntu0.1_amd64.deb ...
Unpacking libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.33.1-1_amd64.deb ...
Unpacking aria2 (1.33.1-1) ...
Setting up libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Setting up aria2 (1.33.1-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3

In [3]:
#@title ### 2. Install dependencies
#@markdown Wait patiently for the installation to complete

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared


/content/naifu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 14.7 MB/s 
     |████████████████████████████████| 468 kB 68.7 MB/s 
+ virtualenv venv
created virtual environment CPython3.7.15.final.0-64 in 890ms
  creator CPython3Posix(dest=/content/naifu/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
+ . venv/bin/activate
++ '[' venv/bin/activate = ./setup.sh ']'
++ deactivate nondestructive
++ unset -f pydoc
++ '[' -z '' ']'
++ '[' -z '' ']'
++ hash -r
++ '[' -z '' ']'
++ unset VIRTUAL_ENV
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/naifu/venv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
100 33.0M  100 33.0M    0     0  5520k      0  0:00:06  0:00:06 --:--:--  344M


In [ ]:
#@title ### 3. Start the model
#@markdown Just visit the output mapping address (end with `trycloudflare.com` / `bore.pub`).
#@markdown - Please wait until the model is loaded (`Application startup complete` appears) before accessing
#@markdown - The service provided by cloudflare occasionally has a request timeout, which can be replaced by bore tunnel

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. (Optional) Run with `animefull-latest` model of 7G
#@markdown uses the 4G size animefull-final-pruned model by default. If you want to use the 7G animefull-latest model, run this

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
# %env MODEL_PATH=models/animefull-latest
%env MODEL_PATH=models
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969


/content
